### Restart and Run All Cells

In [34]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 3, 12)

### Tables in the process

In [35]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

In [36]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [37]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,WHART,20000,12.4,1,DOS,SET,2022-03-17


In [38]:
names = orders['name']
name = names.to_string(index=False)

In [39]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.1195,0.2553,0.1915,0.1915,0.7578,"20,000",2022-03-09,2022-03-25,,0


In [40]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,11.40,I,0.00,12.40,17.84,0.22,12.30,"20,000",-5,4,"20,000",0,0,DOS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [41]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

20

In [42]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AMATA,0,0.00,0,,SET100
1,O,ASIAN,9000,15.20,0,,SET
2,O,ASK,3000,42.00,0,1L,SET
3,O,BAM,10000,20.50,0,,SET100
4,O,BAY,0,0.00,0,,SET
5,O,BBL,2400,130.00,0,1L,SET50
6,O,BDMS,12000,23.20,0,1L,SET50
7,O,CHG,60000,3.36,0,,SET100
8,O,DCC,45000,2.80,0,6L,SET
9,O,EA,0,0.00,0,,SET50


In [43]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,JASIF,10000,11.30,0,DOS,SET
1,I,HREIT,40100,9.00,0,DOS,SET
2,I,RATCH,6000,46.00,0,DOS,SET50
3,I,BCH,5000,21.40,0,CP1S,SET100
4,I,DOHOME,4800,23.30,0,15pct,SET100
5,I,PTT,3000,41.25,0,5pct,SET50
6,I,SINGER,1500,50.75,0,20pct,SET100
7,I,IP,2500,20.30,0,CP1S,mai
8,I,WHART,20000,12.40,0,DOS,SET
9,I,CPNCG,20000,12.50,0,DOS,SET


In [44]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(46, 7)

In [45]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,8,8,8,8,8,8
I,13,13,13,13,13,13
O,25,25,25,25,25,25


In [46]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,8,8,8,8,8,8
O,25,25,25,25,25,25
S,13,13,13,13,13,13


In [47]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(81, 2)

In [48]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(81, 9)

### If there is no dividend record, must create dividend record before proceed

In [49]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge


In [50]:
mask = df_merge['_merge'] == 'both'
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,8,8,8,8,8,8,8
O,25,25,25,25,25,25,25
S,13,13,13,13,13,13,13


### Set xdate 2 weeks window of first Tuesday and second Friday

In [52]:
fm_date = '2022-03-15'
to_date = '2022-04-08'

In [54]:
fltr_sts_o = (df_orders.trade == 'O') & (df_orders.xdate >= fm_date) & (df_orders.xdate <= to_date) 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
8,O,DCC,45000,2.8,0,6L,SET,2022-04-05


In [55]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
14,B,KCE,1000,57.50,0,RD20pct,SET50,2022-03-21
16,B,MAKRO,1500,39.75,0,RD05pct,SET,2022-03-03
17,B,MCS,10000,14.50,0,DOS,SET,2022-03-15
21,B,RCL,3000,48.25,0,DOS,SET100,2022-03-23
25,B,STA,15000,26.25,0,RD20pct,SET100,2022-04-19
28,B,SYNEX,3000,24.00,0,RD15pct,SET100,2022-03-10
29,B,TISCO,500,97.75,0,ROUND,SET50,2022-04-29
30,B,TMT,15000,10.50,0,ROUND,SET,2022-04-18


In [56]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
45,S,AIMIRT,10000,13.10,0,DOS,SET,2022-03-14
43,S,ASP,30000,3.82,0,DOS,SET,2022-03-10
36,S,BCH,5000,21.40,0,CP1S,SET100,2022-05-10
42,S,CPNCG,20000,12.50,0,DOS,SET,2022-02-14
37,S,DOHOME,4800,23.30,0,15pct,SET100,2022-02-02
34,S,HREIT,40100,9.00,0,DOS,SET,2022-03-04
40,S,IP,2500,20.30,0,CP1S,mai,2022-02-02
33,S,JASIF,10000,11.30,0,DOS,SET,2022-03-03
38,S,PTT,3000,41.25,0,5pct,SET50,2022-03-03
35,S,RATCH,6000,46.00,0,DOS,SET50,2022-03-02


In [57]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
14,B,KCE,1000,57.50,0,RD20pct,SET50,2022-03-21
16,B,MAKRO,1500,39.75,0,RD05pct,SET,2022-03-03
17,B,MCS,10000,14.50,0,DOS,SET,2022-03-15
21,B,RCL,3000,48.25,0,DOS,SET100,2022-03-23
25,B,STA,15000,26.25,0,RD20pct,SET100,2022-04-19
28,B,SYNEX,3000,24.00,0,RD15pct,SET100,2022-03-10
29,B,TISCO,500,97.75,0,ROUND,SET50,2022-04-29
30,B,TMT,15000,10.50,0,ROUND,SET,2022-04-18
8,O,DCC,45000,2.80,0,6L,SET,2022-04-05
45,S,AIMIRT,10000,13.10,0,DOS,SET,2022-03-14


In [66]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,9,9,9,9,9,9,9
S,13,13,13,13,13,13,13


In [67]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

In [68]:
rcds = df_orders.values.tolist()
len(rcds)

22

In [69]:
for rcd in rcds:
    print(rcd)

['B', 'KCE', 1000, 57.5, 0, 'RD20pct', 'SET50', '2022-03-21']
['B', 'MAKRO', 1500, 39.75, 0, 'RD05pct', 'SET', '2022-03-03']
['B', 'MCS', 10000, 14.5, 0, 'DOS', 'SET', '2022-03-15']
['B', 'RCL', 3000, 48.25, 0, 'DOS', 'SET100', '2022-03-23']
['B', 'STA', 15000, 26.25, 0, 'RD20pct', 'SET100', '2022-04-19']
['B', 'SYNEX', 3000, 24.0, 0, 'RD15pct', 'SET100', '2022-03-10']
['B', 'TISCO', 500, 97.75, 0, 'ROUND', 'SET50', '2022-04-29']
['B', 'TMT', 15000, 10.5, 0, 'ROUND', 'SET', '2022-04-18']
['B', 'DCC', 45000, 2.8, 0, '6L', 'SET', '2022-04-05']
['S', 'JASIF', 10000, 11.3, 0, 'DOS', 'SET', '2022-03-03']
['S', 'HREIT', 40100, 9.0, 0, 'DOS', 'SET', '2022-03-04']
['S', 'RATCH', 6000, 46.0, 0, 'DOS', 'SET50', '2022-03-02']
['S', 'BCH', 5000, 21.4, 0, 'CP1S', 'SET100', '2022-05-10']
['S', 'DOHOME', 4800, 23.3, 0, '15pct', 'SET100', '2022-02-02']
['S', 'PTT', 3000, 41.25, 0, '5pct', 'SET50', '2022-03-03']
['S', 'SINGER', 1500, 50.75, 0, '20pct', 'SET100', '2022-04-29']
['S', 'IP', 2500, 20.3, 0,

In [70]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [71]:
for rcd in rcds:
    conlite.execute(sql, rcd)

### After call Ord-log.bat

In [72]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [73]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

In [74]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,DCC,-5,6L,45000,2.80,2.90,+0.69%,0,2022-04-05
1,Buy,KCE,-4,RD20pct,1000,57.50,58.50,+0.43%,0,2022-03-21
2,Buy,MAKRO,-7,RD05pct,1500,39.75,41.50,-0.60%,0,2022-03-03
3,Buy,MCS,-2,DOS,10000,14.50,14.70,0.00%,0,2022-03-15
4,Buy,RCL,-7,DOS,3000,48.25,50.00,+1.52%,0,2022-03-23
5,Buy,STA,0,RD20pct,15000,26.25,26.25,+2.94%,0,2022-04-19
6,Buy,SYNEX,-11,RD15pct,3000,24.00,25.25,+2.23%,0,2022-03-10
7,Buy,TISCO,0,ROUND,500,97.75,97.75,0.00%,0,2022-04-29
8,Buy,TMT,-9,ROUND,15000,10.50,11.40,+1.79%,0,2022-04-18
9,Sell,AIMIRT,1,DOS,10000,13.10,13.00,+0.78%,0,2022-03-14
